# Import Libraries 

first , we are going to import the required libraries 

In [ ]:
import pandas as pd
from pandas import DataFrame
import sklearn
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import opinion_lexicon
from sklearn import tree
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import tree
from sklearn.metrics import f1_score
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('opinion_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to /root/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

# Reading Data

after that, we read the CSV file using pandas library 

In [ ]:
data = pd.read_csv("Output.csv", low_memory=False)

In [ ]:
data.Filtered.value_counts()

0.0    95698
1.0    19763
Name: Filtered, dtype: int64

In [ ]:
data = data.head(70000)

handle not a NaN values 

In [ ]:
#reviewerID	reviewContent	rating	usefulCount	date	reviewerID.1	firstCount	reviewCount	Filtered
data['reviewID'] = data['reviewID'].fillna("-1")
data['reviewerID'] = data['reviewerID'].fillna("-1")
data['reviewContent'] = data['reviewContent'].fillna("-1")
data['rating'] = data['rating'].fillna(-1)
data['usefulCount'] = data['usefulCount'].fillna(-1)
data['date'] = data['date'].fillna("-1")
data['firstCount'] = data['firstCount'].fillna(-1)
data['reviewCount'] = data['reviewCount'].fillna(-1)





# Text preprocessing

here we toknize each review content and put each token [list of words] in a list 

In [ ]:
list_of_tokens =[]
for column in data['reviewContent']:
   text_tokens = word_tokenize(column)
   list_of_tokens.append(text_tokens)
len(list_of_tokens)
list_of_tokens

remove stop words and lemmatization on review contents 

In [ ]:
def stopWordRemove(list_of_tokens):
  tokens_without_sw=[]
  sw = set(stopwords.words('english'))
  for i in range(0,len(list_of_tokens)):
    words_list = []
    for word in list_of_tokens[i]:
      # print(word)
      if not (word in sw):
        words_list.append(word)
    tokens_without_sw.append(words_list)
  return tokens_without_sw
tokens_without_sw = stopWordRemove(list_of_tokens)
tokens_without_sw


now we are going to take the output of the *stopWordRemove* and lemmatize the words on it by using the function *lemmatization* 

In [ ]:
def lemmatization(tokens_without_sw):
  lmtz_list=[]
  lemmatizer = WordNetLemmatizer()
  for words_list in tokens_without_sw:
    lmtz_words_list = []
    for word in words_list:
      lmtz_words_list.append(lemmatizer.lemmatize(word))
    lmtz_list.append(lmtz_words_list)
  return  lmtz_list
lmtz_list=lemmatization(tokens_without_sw)
lmtz_list #without sw and lemmatized 

# Feature Extraction 

now we are going to use Bag of words to train our model 
first , we are going to concat the sentances in the *lmtz_list* (stop word removed and lemmatized list) 

In [ ]:
corpus = []
#join the words to use it in vectorizer 

for list_of_words in lmtz_list:
  corpus.append((" ").join(list_of_words))
corpus

after that, we will use *CountVectorizer* to create the bag of words array

In [ ]:
vectorizer = CountVectorizer(max_df=0.90, min_df=0.10)
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names())
X = X.toarray()
# print(X.toarray())

['also', 'always', 'area', 'back', 'bar', 'best', 'come', 'could', 'day', 'even', 'first', 'food', 'friendly', 'get', 'go', 'good', 'got', 'great', 'it', 'know', 'like', 'little', 'lot', 'love', 'make', 'much', 'my', 'nice', 'night', 'one', 'people', 'place', 'pretty', 'price', 'really', 'restaurant', 'right', 'service', 'staff', 'the', 'there', 'they', 'thing', 'this', 'time', 've', 'way', 'we', 'well', 'went', 'would']


**SENTIMENT ANALYSIS TO CLASSIFY EXAGGERATEDOR REVIEWS OR " TOO POSITIVE" AND " TOO NEGATIVE" REVIEWS**
this done by calculating avg poliratity score of each word in a sentance 

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def exaggeratedReviews(lmtz_list):
    sid = SentimentIntensityAnalyzer()
    exaggerated_list =[]
    weight = 0

    for sentance in lmtz_list:
      weight =0 
      for word in sentance: 
        weight = weight+sid.polarity_scores(word)['compound']
      exaggerated_list.append(abs(weight/len(sentance))) #avg weight 
    return exaggerated_list

exaggerated_list = exaggeratedReviews(lmtz_list)

In [ ]:
# len(exaggerated_list)
max(exaggerated_list)

0.6369

here we will divide the senances into 3 subsets 

1.   Too exaggerated 
2.   regular 
3.   not exaggerated 



by using a threshold which is the mean value +- standard diviation 

In [ ]:
exDF = pd.DataFrame(exaggerated_list)
mean = exDF.mean(axis = 0, skipna = True) 
mean[0]

0.036857705947135805

In [ ]:
std = exDF.std(axis = 0, skipna = True)
std

0    0.028297
dtype: float64

In [ ]:
# exDF 
for i in range(0,len(exaggerated_list)):
  if exaggerated_list[i] > (mean[0]+std[0]):
    exaggerated_list[i]= 1
  elif exaggerated_list[i]< (mean[0]-std[0]):
    exaggerated_list[i]=-1
  else:
    exaggerated_list[i]=0 
# exaggerated_list
pd.DataFrame(exaggerated_list)



,0
0,0
1,-1
2,0
3,0
4,-1
...,...
69995,0
69996,0
69997,0
69998,-1


In [ ]:
print(max(exaggerated_list),min(exaggerated_list),0 in exaggerated_list)

1 -1 True


*review content* that contains many symbols such as "<3 < 3 " are often useless and considered spam.


---



In [ ]:

def extractExclamationQuestionMarks(lmtz_list):

  count = 0;  
  numofexclamation = []
  marks = ['!', ">" ,"\'" ,";" ,"\"", "-" ,"?","<"]
  for listt in lmtz_list: 
      count=0
      for word in listt:   
         #Checks whether given character is a punctuation mark  
         if word in marks: 
            count = count + 1  
      numofexclamation.append(count)
  return(numofexclamation)
numofexclamation=extractExclamationQuestionMarks(lmtz_list)
numofexclamation


here we will do cosin similarty , and calculate the avg similarty for each row



---


(can't be run on a data more than 20000) !! ram crashed 





In [ ]:
doc_term_matrix = X.todense()
df = pd.DataFrame(doc_term_matrix, 
                  columns=vectorizer.get_feature_names())
df

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cosSim=cosine_similarity(df, df)
cosSimDF['mean'] = (pd.DataFrame(cosSim)).mean(axis = 1, skipna = True) 
# calculate the mean of each row 
# cosSimDF['mean'] = cosSimDF.mean(axis = 1, skipna = True) 
cosSimDF

In [ ]:
cosSimAVG = cosSimDF['mean'].tolist()
cosSimAVG

In [ ]:
max(cosSimAVG)



---


first, we create new Dataframe for *feature extraction* features with *reviewContents* and *Filtered* columns


In [ ]:
featureData = data[ ['reviewContent','Filtered'] ]

here, we are going to calculate the length of the **review content** using *reviewContentLength* method

In [ ]:
def reviewContentLength(str):
    return len(str)

In [ ]:
featureData['length'] = data.apply( lambda row: reviewContentLength(row['reviewContent']), axis = 1 )
featureData

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,reviewContent,Filtered,length
0,I came to Chicago on business and was initiall...,0.0,767
1,I still love this place but the took Mu Shu of...,0.0,101
2,Alan is the best CPA that I know. He knows all...,0.0,466
3,My family has been using Korean Air for many m...,0.0,791
4,I hadn't been to Grisanti's since a pre-homeco...,0.0,795
...,...,...,...
69995,I'm pretty sure a record skipped as soon as we...,0.0,794
69996,An admitted Parrot Head I couldn't not go to M...,1.0,399
69997,Another failed business. Wasn't terribly sur...,0.0,551
69998,I have visited this location twice -- today an...,0.0,313


In [ ]:
contentLengthList = featureData['length'].to_list()
contentLengthList
len(contentLengthList)

70000

now we catogrize lenghth to 

1.  so tall 1 : when > avg+std 
2.  too short -1 : when < avg+std 
3.  noraml : other wise 



In [ ]:
contentLengthListDF = pd.DataFrame(contentLengthList)
mean = contentLengthListDF.mean(axis = 0, skipna = True) 
mean[0]

546.1968

In [ ]:
std = contentLengthListDF.std(axis = 0, skipna = True)
std

0    243.160685
dtype: float64

In [ ]:
for i in range(0,len(contentLengthList)):
  if contentLengthList[i] > (mean[0]+std[0]):
    contentLengthList[i]= 1
  elif contentLengthList[i]< (mean[0]-std[0]):
    contentLengthList[i]=-1
  else:
    contentLengthList[i]=0 
pd.DataFrame(contentLengthList)

,0
0,0
1,-1
2,0
3,1
4,1
...,...
69995,1
69996,0
69997,0
69998,0


In [ ]:
print(max(contentLengthList),min(contentLengthList),0 in contentLengthList)

1 -1 True


also, we will calculate the **number of times the personal pronouns appears** using *personalPronounsCount* method 

In [ ]:
def personalPronounsCount(lmtz_list):
  '''This function finds the number of personal pronounns like I,Me,myself,mine etc. in the review text'''
  # tokens = nltk.word_tokenize(str)
  pl = []
  for sentance in lmtz_list:
    count=0
    taged = nltk.pos_tag(sentance)
    for i in taged:
      if i[1] == 'PRP' or i[1] == 'PRP$':
        count=count+1
    pl.append(count)
  return pl



In [ ]:
pl =personalPronounsCount(lmtz_list)
pl
# featureData

[4,
 2,
 8,
 3,
 10,
 8,
 0,
 8,
 3,
 5,
 8,
 1,
 6,
 8,
 0,
 10,
 3,
 5,
 0,
 4,
 5,
 8,
 4,
 1,
 3,
 1,
 9,
 2,
 7,
 0,
 9,
 2,
 7,
 0,
 1,
 2,
 5,
 3,
 5,
 2,
 3,
 2,
 3,
 7,
 2,
 3,
 2,
 3,
 1,
 2,
 10,
 2,
 2,
 3,
 5,
 11,
 1,
 3,
 3,
 0,
 4,
 5,
 2,
 5,
 7,
 0,
 7,
 6,
 1,
 4,
 3,
 0,
 0,
 4,
 2,
 2,
 1,
 7,
 2,
 11,
 0,
 4,
 4,
 0,
 0,
 1,
 2,
 3,
 7,
 0,
 14,
 4,
 3,
 8,
 11,
 6,
 7,
 1,
 3,
 7,
 2,
 9,
 10,
 4,
 9,
 4,
 3,
 7,
 2,
 8,
 1,
 0,
 4,
 8,
 4,
 4,
 4,
 2,
 1,
 2,
 1,
 5,
 2,
 10,
 3,
 5,
 2,
 11,
 13,
 5,
 3,
 0,
 2,
 3,
 1,
 3,
 9,
 3,
 1,
 3,
 6,
 5,
 0,
 2,
 6,
 2,
 12,
 5,
 10,
 4,
 2,
 3,
 2,
 5,
 5,
 0,
 3,
 7,
 3,
 7,
 1,
 12,
 9,
 4,
 1,
 5,
 5,
 7,
 3,
 15,
 6,
 15,
 6,
 9,
 10,
 2,
 3,
 5,
 9,
 5,
 6,
 3,
 2,
 2,
 9,
 2,
 9,
 13,
 3,
 4,
 3,
 4,
 5,
 2,
 3,
 12,
 2,
 2,
 10,
 10,
 7,
 0,
 7,
 6,
 2,
 2,
 1,
 2,
 1,
 3,
 0,
 0,
 3,
 5,
 8,
 3,
 3,
 10,
 11,
 2,
 1,
 0,
 8,
 0,
 2,
 1,
 9,
 4,
 12,
 4,
 2,
 10,
 7,
 3,
 6,
 13,
 1,
 6,
 6,
 6,
 0,
 4,
 1,
 1

In [ ]:
personalPronounsCountList = pl
personalPronounsCountList

In [ ]:
len(personalPronounsCountList)

70000

# Classification 

before creating our models, we are going to arrange our fetures in a wanted data structure 

In [ ]:
import numpy as np
X = np.array(X)
# X = np.array(contentLengthList)
X= np.c_[X, contentLengthList]
X = np.c_[X, personalPronounsCountList]
X = np.c_[X, data['rating']]
X = np.c_[X, data['usefulCount']]
X = np.c_[X, data['reviewCount']]
X = np.c_[X, data['firstCount']]
X = np.c_[X , numofexclamation]
# x = np.c_[X , cosSimAVG]



# *************************************************************
# X = np.array(X) # easier to deal with numpy arrays rather thna 2d list 
# X=np.c_[X, contentLengthList] #adding features colums 
# X=np.c_[X, personalPronounsCountList]
# X = np.c_[X, data['rating']]
# X = np.c_[X, data['usefulCount']]
# X = np.c_[X, data['reviewCount']]
# X = np.c_[X, data['firstCount']]
# X = np.c_[X , numofexclamation]

# X
# X.size
#**************************************************************


In [ ]:
len(X) 

70000

#Holdout set 
after that we will divide our dataset into 70% training 30% testing 

In [ ]:
from sklearn.model_selection import train_test_split
X= X.tolist()
Y = data['Filtered'].tolist()
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.30 )


Handle NaN data 

In [ ]:

X_train = np.nan_to_num(np.array(X_train))
X_test = np.nan_to_num(np.array(X_test))
Y_train = np.nan_to_num(np.array(Y_train))
Y_test = np.nan_to_num(np.array(Y_test))

In [ ]:
X_train=np.array(X_train).astype(np.float)
X_test=np.array(X_test).astype(np.float)
Y_train=np.array(Y_train).astype(np.float)
Y_test=np.array(Y_test).astype(np.float)

Decision Trees (DT) 

In [ ]:
from sklearn import tree

clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, Y_train)
# clf.max_depth()

In [ ]:

count=0
predicted_out=clf.predict(X_test)
print(accuracy_score(Y_test, predicted_out),f1_score(Y_test, predicted_out),precision_score(Y_test, predicted_out), recall_score(Y_test, predicted_out))

0.7715714285714286 0.35324255089658896 0.3352098259979529 0.3733257338273012


Gaussian Naive Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb_model = gnb.fit(X_train, Y_train)
gnb_model

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
count=0
predicted_out=gnb_model.predict(X_test)
print(accuracy_score(Y_test, predicted_out),f1_score(Y_test, predicted_out),precision_score(Y_test, predicted_out), recall_score(Y_test, predicted_out))

# for i in range (0,len(Y_test)):
#    if predicted_out[i] == Y_test[i]:
#           count+=1
# print(count)

0.6349047619047619 0.3226433430515063 0.23380281690140844 0.5203761755485894


Neural netweork multi layer perceptron 

In [ ]:
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(3, 3), random_state=1)

mlp_model.fit(X_train, Y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 3), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
count=0
predicted_out=mlp_model.predict(X_test)
print(accuracy_score(Y_test, predicted_out),f1_score(Y_test, predicted_out),precision_score(Y_test, predicted_out), recall_score(Y_test, predicted_out))



0.17704761904761904 0.27563081565931763 0.16158042164234115 0.9370190937589057


# NOW WE ARE GOING TO BALNCE THE DATA !! using smote

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE

SMOTE = SMOTE()
X_train_SMOTE, Y_train_SMOTE = SMOTE.fit_resample(X_train, Y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
print("befor balncing: "+str(Counter(Y_train)),"after balancing: "+ str(Counter(Y_train_SMOTE)))

befor balncing: Counter({0.0: 40622, 1.0: 8378}) after balancing: Counter({0.0: 40622, 1.0: 40622})


now we will apply the same classifiers 

In [ ]:
from sklearn import tree

DT = tree.DecisionTreeClassifier()
DT = DT.fit(X_train_SMOTE, Y_train_SMOTE)
gnb = GaussianNB()
gnb_model = gnb.fit(X_train_SMOTE, Y_train_SMOTE)
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(3, 3), random_state=1)

mlp_model=mlp_model.fit(X_train_SMOTE, Y_train_SMOTE)


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


now we will see the results 

In [ ]:
DT_predicted=clf.predict(X_test)
print(accuracy_score(Y_test, DT_predicted),f1_score(Y_test, DT_predicted),precision_score(Y_test, DT_predicted), recall_score(Y_test, DT_predicted))


0.7715714285714286 0.35324255089658896 0.3352098259979529 0.3733257338273012


In [ ]:
predicted_out=gnb_model.predict(X_test)
print(accuracy_score(Y_test, predicted_out),f1_score(Y_test, predicted_out),precision_score(Y_test, predicted_out), recall_score(Y_test, predicted_out))


0.5708571428571428 0.30063634952661805 0.206569265223419 0.5520091194072385


In [ ]:
predicted_out=mlp_model.predict(X_test)
print(accuracy_score(Y_test, predicted_out),f1_score(Y_test, predicted_out),precision_score(Y_test, predicted_out), recall_score(Y_test, predicted_out))


0.8500952380952381 0.3744038155802861 0.618516086671044 0.268452550584212
